In [1]:
!pip install  pytorch_transformers

fastai 1.0.55 requires nvidia-ml-py3, which is not installed.
thinc 6.12.1 has requirement msgpack<0.6.0,>=0.5.6, but you'll have msgpack 0.6.0 which is incompatible.
You are using pip version 10.0.1, however version 19.2.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [ ]:
import sagemaker

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = 'compliance-data/batch'

role = sagemaker.get_execution_role()

In [ ]:
bucket

In [ ]:
!ls /home/ec2-user/SageMaker/transformers-classification/data

In [ ]:
inputs = sagemaker_session.upload_data(path='/home/ec2-user/SageMaker/transformers-classification/data', bucket=bucket, key_prefix=prefix)
print('input spec (in this case, just an S3 path): {}'.format(inputs))

In [2]:
import json
config_json='/home/ec2-user/SageMaker/transformers-classification/config-bert.json'
print("using config "+config_json)
with open(config_json, 'r') as f:
    args= json.load(f)

print(args)

using config /home/ec2-user/SageMaker/transformers-classification/config-bert.json
{'data_dir': '../data/', 'model_type': 'bert', 'model_name': 'bert-base-uncased', 'task_name': 'binary', 'output_dir': '../outputs_bert/', 'cache_dir': 'cache/', 'do_train': True, 'do_eval': True, 'fp16': False, 'fp16_opt_level': 'O1', 'max_seq_length': 512, 'output_mode': 'classification', 'train_batch_size': 48, 'eval_batch_size': 12, 'gradient_accumulation_steps': 1, 'num_train_epochs': 8, 'weight_decay': 0, 'learning_rate': 4e-05, 'adam_epsilon': 1e-08, 'warmup_steps': 0, 'max_grad_norm': 1.0, 'logging_steps': 50, 'evaluate_during_training': False, 'save_steps': 1000, 'eval_all_checkpoints': True, 'overwrite_output_dir': True, 'reprocess_input_data': False, 'notes': 'first batch of compliance data'}


In [3]:
args['num_train_epochs']=1
args['data_dir']='/home/ec2-user/SageMaker/transformers-classification/data'
args['model_dir']='/home/ec2-user/SageMaker/transformers-classification/models'
args['save_steps']=200
args['num_train_epochs']=1

# start of python program

In [4]:
from __future__ import absolute_import, division, print_function

import ast
import glob
import logging
import os
import sys
import time
import random
import json

import numpy as np
import torch
import argparse
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,
                              TensorDataset)
import random
from torch.utils.data.distributed import DistributedSampler
from tqdm import tqdm_notebook, trange


from pytorch_transformers import (WEIGHTS_NAME, BertConfig, BertForSequenceClassification, BertTokenizer,
                                  XLMConfig, XLMForSequenceClassification, XLMTokenizer, 
                                  XLNetConfig, XLNetForSequenceClassification, XLNetTokenizer,
                                  RobertaConfig, RobertaForSequenceClassification, RobertaTokenizer)

from pytorch_transformers import AdamW, WarmupLinearSchedule

from utils import (convert_examples_to_features,
                        output_modes, processors)


logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [5]:
# globals

MODEL_CLASSES = {
    'bert': (BertConfig, BertForSequenceClassification, BertTokenizer),
    'xlnet': (XLNetConfig, XLNetForSequenceClassification, XLNetTokenizer),
    'xlm': (XLMConfig, XLMForSequenceClassification, XLMTokenizer),
    'roberta': (RobertaConfig, RobertaForSequenceClassification, RobertaTokenizer)
}


In [ ]:
def load_and_cache_examples(args, tokenizer, evaluate=False):
    
    output_mode = args['output_mode']
    task = args['task_name']
    processor = processors[task]()
    
    mode = 'dev' if evaluate else 'train'
    cached_features_file = os.path.join(args['data_dir'], f"cached_{mode}_{args['model_name']}_{args['max_seq_length']}_{task}")
    
    if os.path.exists(cached_features_file) and not args['reprocess_input_data']:
        logger.info("Loading features from cached file %s", cached_features_file)
        features = torch.load(cached_features_file)
               
    else:
        logger.info("Creating features from dataset file at %s", args['data_dir'])
        label_list = processor.get_labels()
        examples = processor.get_dev_examples(args['data_dir']) if evaluate else processor.get_train_examples(args['data_dir'])
        
        features = convert_examples_to_features(examples, label_list, args['max_seq_length'], tokenizer, output_mode,
            cls_token_at_end=bool(args['model_type'] in ['xlnet']),            # xlnet has a cls token at the end
            cls_token=tokenizer.cls_token,
            sep_token=tokenizer.sep_token,
            cls_token_segment_id=2 if args['model_type'] in ['xlnet'] else 0,
            pad_on_left=bool(args['model_type'] in ['xlnet']),                 # pad on the left for xlnet
            pad_token_segment_id=4 if args['model_type'] in ['xlnet'] else 0)
        
        logger.info("Saving features into cached file %s", cached_features_file)
        torch.save(features, cached_features_file)
        
    all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
    all_input_mask = torch.tensor([f.input_mask for f in features], dtype=torch.long)
    all_segment_ids = torch.tensor([f.segment_ids for f in features], dtype=torch.long)
    all_label_ids = torch.tensor([f.label_id for f in features], dtype=torch.long)

    dataset = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)
    return dataset

In [ ]:
from sklearn.metrics import mean_squared_error, matthews_corrcoef, confusion_matrix
from scipy.stats import pearsonr

def get_mismatched(labels, preds):
    mismatched = labels != preds
    
    processor = processors['binary']()
    examples = processor.get_dev_examples(args['data_dir'])
    wrong = [i for (i, v) in zip(examples, mismatched) if v]
    
    return wrong

def get_eval_report(labels, preds):
    mcc = matthews_corrcoef(labels, preds)
    tn, fp, fn, tp = confusion_matrix(labels, preds).ravel()
    return {
        "matthews_corrcoef": mcc,
        "tp": tp,
        "tn": tn,
        "fp": fp,
        "fn": fn,
        "accuracy": float(tp+tn)/(tp+tn+fp+fn),
        "f1": float(2*tp)/(2*tp+fp+fn)
    }, get_mismatched(labels, preds)

def compute_metrics(task_name, preds, labels):
    assert len(preds) == len(labels)
    return get_eval_report(labels, preds)

def evaluate(model, tokenizer, prefix=""):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Loop to handle MNLI double evaluation (matched, mis-matched)
    eval_output_dir = args['output_dir']

    results = {}
    EVAL_TASK = args['task_name']

    eval_dataset = load_and_cache_examples(args, tokenizer, evaluate=True)
    if not os.path.exists(eval_output_dir):
        os.makedirs(eval_output_dir)


    eval_sampler = SequentialSampler(eval_dataset)
    eval_dataloader = DataLoader(eval_dataset, sampler=eval_sampler, batch_size=args['eval_batch_size'])

    # Eval!
    logger.info("***** Running evaluation {} *****".format(prefix))
    logger.info("  Num examples = %d", len(eval_dataset))
    logger.info("  Batch size = %d", args['eval_batch_size'])
    eval_loss = 0.0
    nb_eval_steps = 0
    preds = None
    out_label_ids = None
    for batch in tqdm_notebook(eval_dataloader, desc="Evaluating"):
        model.eval()
        batch = tuple(t.to(device) for t in batch)

        with torch.no_grad():
            inputs = {'input_ids':      batch[0],
                      'attention_mask': batch[1],
                      'token_type_ids': batch[2] if args['model_type'] in ['bert', 'xlnet'] else None,  # XLM don't use segment_ids
                      'labels':         batch[3]}
            outputs = model(**inputs)
            tmp_eval_loss, logits = outputs[:2]

            eval_loss += tmp_eval_loss.mean().item()
        nb_eval_steps += 1
        if preds is None:
            preds = logits.detach().cpu().numpy()
            out_label_ids = inputs['labels'].detach().cpu().numpy()
        else:
            preds = np.append(preds, logits.detach().cpu().numpy(), axis=0)
            out_label_ids = np.append(out_label_ids, inputs['labels'].detach().cpu().numpy(), axis=0)

    eval_loss = eval_loss / nb_eval_steps
    if args['output_mode'] == "classification":
        preds = np.argmax(preds, axis=1)
    elif args['output_mode'] == "regression":
        preds = np.squeeze(preds)
    result, wrong = compute_metrics(EVAL_TASK, preds, out_label_ids)
    results.update(result)

    output_eval_file = os.path.join(eval_output_dir, "eval_results.txt")
    with open(output_eval_file, "a") as writer:
        logger.info("***** Eval results {} *****".format(prefix))
        writer.write("***** Eval results {} *****\n".format(prefix))
        for key in sorted(result.keys()):
            logger.info("  %s = %s", key, str(result[key]))
            writer.write("%s = %s\n" % (key, str(result[key])))

    return results

In [ ]:
def _train(args):
    # initialization
    config_class, model_class, tokenizer_class = MODEL_CLASSES[args['model_type']]

    config = config_class.from_pretrained(args['model_name'], num_labels=2, finetuning_task=args['task_name'])
    tokenizer = tokenizer_class.from_pretrained(args['model_name'])
    model = model_class.from_pretrained(args['model_name'])
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    if torch.cuda.device_count() > 1:
        print("Training: use", torch.cuda.device_count(), "GPUs!")
        model = torch.nn.DataParallel(model)
    model.to(device)
    
    logger.info("Loading  dataset")
    train_dataset= load_and_cache_examples(args, tokenizer, False)
                   
    train_sampler = RandomSampler(train_dataset)
    train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=args['train_batch_size'])
    print("len(train_dataloader) "+ str(len(train_dataloader))) 
    t_total = len(train_dataloader) // args['gradient_accumulation_steps'] * args['num_train_epochs']
    
    no_decay = ['bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': args['weight_decay']},
        {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=args['learning_rate'], eps=args['adam_epsilon'])
    scheduler = WarmupLinearSchedule(optimizer, warmup_steps=args['warmup_steps'], t_total=t_total)
            
    logger.info("***** Running training *****")
    logger.info("  Num examples = %d", len(train_dataset))
    logger.info("  Num Epochs = %d", args['num_train_epochs'])
    logger.info("  Total train batch size  = %d", args['train_batch_size'])
    logger.info("  Gradient Accumulation steps = %d", args['gradient_accumulation_steps'])
    logger.info("  Total optimization steps = %d", t_total)

    global_step = 0
    tr_loss, logging_loss = 0.0, 0.0
    model.zero_grad()
    train_iterator = trange(int(args['num_train_epochs']), desc="Epoch")
   
    
    for _ in train_iterator:
        epoch_iterator = tqdm_notebook(train_dataloader, desc="Iteration")
        for step, batch in enumerate(epoch_iterator):
            model.train()
            batch = tuple(t.to(device) for t in batch)
            inputs = {'input_ids':      batch[0],
                      'attention_mask': batch[1],
                      'token_type_ids': batch[2] if args['model_type'] in ['bert', 'xlnet'] else None,  # XLM don't use segment_ids
                      'labels':         batch[3]}
            outputs = model(**inputs)
            loss = outputs[0].mean()  # model outputs are always tuple in pytorch-transformers (see doc)
            print("\r%f" % loss, end='')

            if args['gradient_accumulation_steps'] > 1:
                loss = loss / args['gradient_accumulation_steps']

            if args['fp16']:
                with amp.scale_loss(loss, optimizer) as scaled_loss:
                    scaled_loss.backward()
                torch.nn.utils.clip_grad_norm_(amp.master_params(optimizer), args['max_grad_norm'])
                
            else:
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), args['max_grad_norm'])

            tr_loss += loss.item()
            if (step + 1) % args['gradient_accumulation_steps'] == 0:
                scheduler.step()  # Update learning rate schedule
                optimizer.step()
                model.zero_grad()
                global_step += 1

                if args['logging_steps'] > 0 and global_step % args['logging_steps'] == 0:
                     
                    logging_loss = tr_loss

                if args['save_steps'] > 0 and global_step % args['save_steps'] == 0:
                    # Save model checkpoint
                    output_dir = os.path.join(args['output_dir'], 'checkpoint-{}'.format(global_step))
                    if not os.path.exists(output_dir):
                        os.makedirs(output_dir)
                    model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
                    model_to_save.save_pretrained(output_dir)
                    logger.info("Saving model checkpoint to %s", output_dir)
                    
                    
    logger.info("starting evaluating ")
    checkpoints = list(os.path.dirname(c) for c in sorted(glob.glob(args['output_dir'] + '/**/' + WEIGHTS_NAME, recursive=True)))
    logging.getLogger("pytorch_transformers.modeling_utils").setLevel(logging.WARN)  # Reduce logging
    logger.info("Evaluate the following checkpoints: %s", checkpoints)
    best_result = None
    best_checkpoint = None
    results=[]
    for checkpoint in checkpoints:
        global_step = checkpoint.split('-')[-1] if len(checkpoints) > 1 else ""
        model = model_class.from_pretrained(checkpoint)
        model.to(device)
        result = evaluate(model, tokenizer, prefix=global_step)
        
        logger.info(" result,{%s}", result)
        if best_result is None or result['matthews_corrcoef'] > best_result['matthews_corrcoef']:
            best_result = result
            best_checkpoint= checkpoint
            logger.info("best result, Saving model checkpoint to %s", best_checkpoint)
            
        result = dict((k + '_{}'.format(global_step), v) for k, v in result.items())
        results.append(result) 
    # save best model
    model = model_class.from_pretrained(best_checkpoint) 
    model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
    model_to_save.save_pretrained(args['model_dir'])
 

In [ ]:
if __name__ == '__main__':
    parser = argparse.ArgumentParser()

    parser.add_argument('--workers', type=int, default=2, metavar='W',
                        help='number of data loading workers (default: 2)')
    parser.add_argument('--num_train_epochs', type=int, default=4, metavar='E',
                        help='number of total epochs to run (default: 4)')
    parser.add_argument('--train_batch_size', type=int, default=48, metavar='TBS',
                        help='train batch size, can be signle or multiple GPUs (default: 48)')
    parser.add_argument('--eval_batch_size', type=int, default=12, metavar='EBS',
                        help='eval batch size, on single GPU,(default: 12)')
    parser.add_argument('--weight_decay', type=int, default=0, metavar='WD',
                        help='initial weight_decay (default: 0)')
    parser.add_argument('--learning_rate', type=float, default=4e-05, metavar='LR',
                        help='initial learning rate (default: 4e-05)')
    parser.add_argument('--adam_epsilon', type=float, default=1e-08, 
                        help='initial adam_epsilon (default: 1e-08)')
    parser.add_argument('--warmup_steps', type=int, default=0, 
                        help='initial warmup_steps (default: 0)')
    parser.add_argument('--max_grad_norm', type=float, default=1.0,
                        help='max_grad_norm (default: 1.0)')
   
    parser.add_argument('--model_type', type=str, default='bert')
    parser.add_argument('--model_name', type=str, default='bert-base-uncased')
    parser.add_argument('--task_name', type=str, default='binary')
    parser.add_argument('--output_mode', type=str, default='classification')
    parser.add_argument('--max_seq_length', type=int, default=512)
    
    parser.add_argument('--fp16', type=bool, default=False)
    parser.add_argument('--fp16_opt_level', type=str, default='O1')

    parser.add_argument('--gradient_accumulation_steps', type=int, default=1)
    parser.add_argument('--logging_steps', type=int, default=500)
    parser.add_argument('--save_steps', type=int, default=886)
    
    parser.add_argument('--reprocess_input_data', type=bool, default=False)
    
    # The parameters below retrieve their default values from SageMaker environment variables, which are
    # instantiated by the SageMaker containers framework.
    # https://github.com/aws/sagemaker-containers#how-a-script-is-executed-inside-the-container
    
    #parser.add_argument('--hosts', type=str, default=ast.literal_eval(os.environ['SM_HOSTS']))
    #parser.add_argument('--current-host', type=str, default=os.environ['SM_CURRENT_HOST'])
    #parser.add_argument('--model-dir', type=str, default=os.environ['SM_MODEL_DIR'])
    #parser.add_argument('--data-dir', type=str, default=os.environ['SM_CHANNEL_TRAINING'])
    #parser.add_argument('--num-gpus', type=int, default=os.environ['SM_NUM_GPUS'])
    parser.add_argument('--model-dir', type=str, default='/home/ec2-user/SageMaker/transformers-classification/models')
    parser.add_argument('--data-dir', type=str, default='/home/ec2-user/SageMaker/transformers-classification/data')
    parser.add_argument('--output-dir', type=str, default='/home/ec2-user/SageMaker/transformers-classification/outputs')
    
    parser.add_argument('-f', type=str, default='/home/ec2-user/SageMaker/transformers-classification/outputs')
    args= vars(parser.parse_args())
    print(parser.parse_args())
    _train(args)

In [27]:
import uuid
from utils import InputExample, convert_example_to_feature
JSON_CONTENT_TYPE = 'application/json'
model_type = args['model_type']
label_map = {'0': 0, '1': 1}
output_mode='classification'

# intialized config and tokenizer, as it is used by both training and inference
config_class, model_class, tokenizer_class = MODEL_CLASSES[args['model_type']]

config = config_class.from_pretrained(args['model_name'], num_labels=2, finetuning_task=args['task_name'])
tokenizer = tokenizer_class.from_pretrained(args['model_name'])

INFO:pytorch_transformers.modeling_utils:loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /home/ec2-user/.cache/torch/pytorch_transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.bf3b9ea126d8c0001ee8a1e8b92229871d06d36d8808208cc2449280da87785c
INFO:pytorch_transformers.modeling_utils:Model config {
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": "binary",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "pruned_heads": {},
  "torchscript": false,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

INFO:pytorch_transformers.tokenization_utils:loading file https://s3.amazonaws.com/models.huggingface.c

In [42]:
def model_fn(model_dir):
    logger.info('model_fn')
       
    # initialization
    config_class, model_class, tokenizer_class = MODEL_CLASSES[args['model_type']]

    config = config_class.from_pretrained(args['model_name'], num_labels=2, finetuning_task=args['task_name'])
    tokenizer = tokenizer_class.from_pretrained(args['model_name'])
    model = model_class.from_pretrained(model_dir)
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    if torch.cuda.device_count() > 1:
        logger.info("Gpu count: {}".format(torch.cuda.device_count()))
        model = torch.nn.DataParallel(model)

    return model.to(device)

In [43]:
def input_fn(serialized_input_data, content_type=JSON_CONTENT_TYPE):
    logger.info('Deserializing the input data.')
    if content_type == JSON_CONTENT_TYPE:
        input_data = json.loads(serialized_input_data)
        return input_data
    raise Exception('Requested unsupported ContentType in content_type: ' + content_type)


def output_fn(prediction_output, accept=JSON_CONTENT_TYPE):
    logger.info('Serializing the generated output.')
    if accept == JSON_CONTENT_TYPE:
        return json.dumps(prediction_output), accept
    raise Exception('Requested unsupported ContentType in Accept: ' + accept)

In [45]:
def predict_fn(input_data, model):
   
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    example=InputExample(str(uuid.uuid1()), input_data, None, '0')
    print("exameple label",example.label)
    
    cls_token_at_end=bool(model_type in ['xlnet'])           # xlnet has a cls token at the end
    cls_token=tokenizer.cls_token,
    sep_token=tokenizer.sep_token,
    cls_token_segment_id=2 if model_type in ['xlnet'] else 0
    pad_on_left=bool(model_type in ['xlnet'])               # pad on the left for xlnet
    pad_token_segment_id=4 if model_type in ['xlnet'] else 0
    
    example_row =  (example, label_map, 512, tokenizer, output_mode, cls_token_at_end, cls_token, sep_token, cls_token_segment_id, pad_on_left, pad_token_segment_id, False)
    f = convert_example_to_feature(example_row)
    #print(features)    
    all_input_ids = torch.tensor([f.input_ids ], dtype=torch.long)
    all_input_mask = torch.tensor([f.input_mask ], dtype=torch.long)
    all_segment_ids = torch.tensor([f.segment_ids ], dtype=torch.long)
    all_label_ids = torch.tensor([f.label_id ], dtype=torch.long)

     
    model.eval()
    with torch.no_grad():
        inputs = {'input_ids':      all_input_ids,
                      'attention_mask': all_input_mask,
                      'token_type_ids': all_segment_ids if args['model_type'] in ['bert', 'xlnet'] else None,  # XLM don't use segment_ids
                      'labels':         all_label_ids }
        outputs = model(**inputs)
        _, logits = outputs[:2]

    preds = logits.detach().cpu().numpy()
             
    preds = np.argmax(preds, axis=1)
    print(preds[0])
    return str(preds[0])
    #return '\n'.join(preds)

In [46]:
input_data='Happy birthday, Mike. It is your special day and you deserve a special gift. Enjoy and have fun'
model=model_fn('/home/ec2-user/SageMaker/transformers-classification/models')
start_t=time.time()
predict_fn(input_data, model)
time.time()-start_t

INFO:__main__:model_fn
INFO:pytorch_transformers.modeling_utils:loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /home/ec2-user/.cache/torch/pytorch_transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.bf3b9ea126d8c0001ee8a1e8b92229871d06d36d8808208cc2449280da87785c
INFO:pytorch_transformers.modeling_utils:Model config {
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": "binary",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "pruned_heads": {},
  "torchscript": false,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

INFO:pytorch_transformers.tokenization_utils:loading file https://s3.amazonaws.c

exameple label 0
0


/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/nn/parallel/_functions.py:61: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


0.031184673309326172

In [ ]:
label_map[example.label]